In [1]:
import requests
from bs4 import BeautifulSoup
import re
from unidecode import unidecode
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

from selenium.common.exceptions import NoSuchElementException

In [2]:
chromedriver = '/Users/alex/Downloads/chromedriver'
browser = webdriver.Chrome(chromedriver)
url = 'https://reverb.com/marketplace/keyboards-and-synths?show_only_sold=true'
browser.get(url)
try:
    myElem = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.NAME, 'grid-card__inner')))
    print "Page is ready!"
except TimeoutException:
    print "Loading took too much time!"

respData = browser.page_source
soup = BeautifulSoup(respData, 'html.parser')

Loading took too much time!


In [13]:
brands = []
conditions = []
descriptions = []
models = []
types = []
prices = []
year_made = []
made_in = []
n_keys = []
urls = []

page_count = 0

while page_count <= 50:
    
    time.sleep(2)
    
    respData = browser.page_source
    soup = BeautifulSoup(respData, 'html.parser')
    
    synth_links = soup.find_all('a', attrs={'class': 'grid-card__inner'})

    for i in range(len(synth_links)):
        browser.get(synth_links[i]['href'])
        element_1 = browser.find_element_by_xpath('//*[@id="description"]/div[2]/div[1]')
        element_1.click()

        try:
            element_2 = browser.find_element_by_xpath(
                '//*[@id="description"]/div[2]/div[2]/table/tbody/tr[7]/td/button')
            element_2.click()
        except NoSuchElementException:
            pass

        try:
            element_button_3 = browser.find_element_by_xpath('//*[@id="description"]/div[1]/div[2]/div[1]/button')
            element_button_3.click()
        except NoSuchElementException:
            pass
        
        respData = browser.page_source
        soup = BeautifulSoup(respData, 'html.parser')

        start_time = time.time()

        try:
            price = soup.find('span', attrs={'class': 'price-display'}).text   
        except IndexError:
            price = 'N/A'

        try:
            condition = soup.find_all(
                'tr', attrs={'data-condition': "true"})[0].text.split('(')[0].replace('Condition', '').strip()
        except IndexError:
            condition = 'N/A'

        remaining_specs = browser.find_element_by_css_selector('tbody.collapsing-list').text.split('\n')
        
        for x in range(len(remaining_specs)):
            if remaining_specs[x] == 'Brand':
                brand = remaining_specs[x+1]
            if 'Brand' not in remaining_specs:
                brand = 'N/A'
                
            if remaining_specs[x] == 'Model':
                model = remaining_specs[x+1]
            if 'Model' not in remaining_specs:
                model = 'N/A'
                
            if remaining_specs[x] == 'Categories':
                inst_type = remaining_specs[x+1]
            if 'Categories' not in remaining_specs:
                inst_type = 'N/A'
            
            if remaining_specs[x] == 'Year':
                year = remaining_specs[x+1]
            if 'Year' not in remaining_specs:
                year = 'N/A'
                
            if remaining_specs[x]  == 'Made In':
                made_in_ = remaining_specs[x+1]
            if 'Made In' not in remaining_specs:
                made_in_ = 'N/A'
                
            if remaining_specs[x] == 'Number of Keys':
                n_keys_ = remaining_specs[x+1]
            if 'Number of Keys' not in remaining_specs:
                n_keys_ = 0

        try:
            description = browser.find_element_by_xpath('//*[@id="description"]/div[1]/div[2]').text
            description = description.replace('Payment & Returns', '')
            description = description.replace('\n', ' ')
        except NoSuchElementException:
            description = 'N/A'


        brands.append(brand)
        models.append(model)
        types.append(inst_type)
        year_made.append(year)
        made_in.append(made_in_)
        n_keys.append(n_keys_)
        prices.append(price)
        conditions.append(condition)
        descriptions.append(description)
        urls.append(synth_links[i]['href'])

        time.sleep(1)

        #print('Elapsed Time: {} seconds'.format(time.time() - start_time))
        if i % 24 == 0:
            print('24 more instruments added')
        browser.back()
    
    time.sleep(3)
    browser.find_element_by_link_text("Next").click()
    page_count += 1

24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more instruments added
24 more inst

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="description"]/div[2]/div[1]"}
  (Session info: chrome=81.0.4044.122)


In [14]:
df = pd.DataFrame(data=[brands, conditions, descriptions, models, types, prices, year_made, made_in, n_keys, urls],
                 index=['Brand', 'Condition', 'Description', 'Model', 'synth_types', 'Price',
                         'year_made', 'made_in', 'n_keys', 'urls'])
df = df.transpose()
df

,Brand,Condition,Description,Model,synth_types,Price,year_made,made_in,n_keys,urls
0,M-Audio,Excellent,Keyboard controller is in very good condition ...,Oxygen 49 MkIV USB MIDI Keyboard Controller,Controllers,$99,2010s,United States,49 Keys,https://reverb.com/item/33222243-m-audio-oxyge...
1,Alesis,Mint,It’s open box but never been used so the condi...,V61 USB MIDI Controller,Controllers,$140,2010s,N/A,61 Keys,https://reverb.com/item/33222159-alesis-v61-us...
2,Polyend,Mint,"Like new, awesome collab between Polyend + Dre...",Medusa,Analog Synths,$699,2018,N/A,0,https://reverb.com/item/33222113-polyend-dread...
3,Expert Sleepers,Excellent,"Expert Sleepers in perfect condition, Includes...",ES-8 USB Audio Interface,Modular Synths,$350,2010s,United Kingdom,0,https://reverb.com/item/33222045-expert-sleepe...
4,Behringer,Mint,Purchased new 6 months ago. Instrument has not...,DeepMind 12 49-Key 12-Voice Polyphonic Analog ...,Analog Synths,$625,2016,N/A,49 Keys,https://reverb.com/item/33221691-behringer-dee...
5,Roland,Mint,"Used JU-06A in mint condition, very lightly us...",Boutique Series JU-06A Synthesizer Module,Digital Synths,$365,2019,China,0,https://reverb.com/item/33221717-roland-boutiq...
6,Random Source,Mint,For sale is a MINT Serge DUSG by Random Source...,Dual Universal Slope Generator,Eurorack,$420,2018,United States,0,https://reverb.com/item/33221601-random-source...
7,Korg,Excellent,The classic MS-20 with Moog skin ($75 value). ...,MS-20 Mini Semi-Modular Analog Synthesizer,Analog Synths,$380,2010s,N/A,37 Keys,https://reverb.com/item/33221571-korg-ms-20-mi...
8,Critter & Guitari,Excellent,Looking to drive away that quarantine insanity...,Organelle,Digital Synths,$430,2018,N/A,0,https://reverb.com/item/33221618-critter-guita...
9,Critter & Guitari,Excellent,A powerful and flexible music computer. It's o...,Organelle M,Digital Synths,$500,2019,United States,0,https://reverb.com/item/33221514-critter-guita...


In [15]:
df.to_csv('April_23rd_synth_sold_data_5.csv', encoding='utf-8')